<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#导入相关库" data-toc-modified-id="导入相关库-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>导入相关库</a></span></li><li><span><a href="#基础设置" data-toc-modified-id="基础设置-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>基础设置</a></span></li><li><span><a href="#预处理" data-toc-modified-id="预处理-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>预处理</a></span></li><li><span><a href="#设置训练集&amp;测试集迭代器" data-toc-modified-id="设置训练集&amp;测试集迭代器-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>设置训练集&amp;测试集迭代器</a></span></li><li><span><a href="#训练" data-toc-modified-id="训练-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>训练</a></span></li><li><span><a href="#保存模型" data-toc-modified-id="保存模型-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>保存模型</a></span></li></ul></div>

# 导入相关库

In [3]:
from keras.models import Sequential
from keras.layers import Conv2D,MaxPool2D,Activation,Dropout,Flatten,Dense
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator,img_to_array,load_img
from keras.models import load_model

D:\soft\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# 基础设置

In [11]:
IMG_W = 224 #定义裁剪的图片宽度
IMG_H = 224 #定义裁剪的图片高度
CLASS = 2 #图片的分类数
EPOCHS = 50 #迭代周期
BATCH_SIZE = 64 #批次大小
LEARNING_RATE = 1e-4 #学习率
DROPOUT_RATE = 0 #抗拟合，不工作的神经网络百分比

In [5]:
TRAIN_PATH = 'train' #训练集存放路径
TEST_PATH = 'test' #测试集存放路径
SAVE_PATH = 'Model' #模型保存路径

# 预处理

In [6]:
train_datagen = ImageDataGenerator(
    rotation_range = 40,     # 随机旋转度数
    width_shift_range = 0.2, # 随机水平平移
    height_shift_range = 0.2,# 随机竖直平移
    rescale = 1/255,         # 数据归一化
    shear_range = 20,       # 随机错切变换
    zoom_range = 0.2,        # 随机放大
    horizontal_flip = True,  # 水平翻转
    fill_mode = 'nearest',   # 填充方式
)
test_datagen = ImageDataGenerator(
    rescale = 1/255,         # 数据归一化
)

model = Sequential() #创建一个神经网络对象

In [7]:
#添加一个卷积层，传入固定宽高三通道的图片，以32种不同的卷积核构建32张特征图，
# 卷积核大小为3*3，构建特征图比例和原图相同，激活函数为relu函数。
model.add(Conv2D(input_shape=(IMG_W,IMG_H,3),filters=32,kernel_size=3,padding='same',activation='relu'))
#再次构建一个卷积层
model.add(Conv2D(filters=32,kernel_size=3,padding='same',activation='relu'))
#构建一个池化层，提取特征，池化层的池化窗口为2*2，步长为2。
model.add(MaxPool2D(pool_size=2,strides=2))
#继续构建卷积层和池化层，区别是卷积核数量为64。
model.add(Conv2D(filters=64,kernel_size=3,padding='same',activation='relu'))
model.add(Conv2D(filters=64,kernel_size=3,padding='same',activation='relu'))
model.add(MaxPool2D(pool_size=2,strides=2))
#继续构建卷积层和池化层，区别是卷积核数量为128。
model.add(Conv2D(filters=128,kernel_size=3,padding='same',activation='relu'))
model.add(Conv2D(filters=128,kernel_size=3,padding='same',activation='relu'))
model.add(MaxPool2D(pool_size=2, strides=2))

model.add(Flatten()) #数据扁平化
model.add(Dense(128,activation='relu')) #构建一个具有128个神经元的全连接层
model.add(Dense(64,activation='relu')) #构建一个具有64个神经元的全连接层
model.add(Dropout(DROPOUT_RATE)) #加入dropout，防止过拟合。
model.add(Dense(CLASS,activation='softmax')) #输出层，一共14个神经元，对应14个分类

adam = Adam(lr=LEARNING_RATE) #创建Adam优化器

model.compile(optimizer=adam,loss='categorical_crossentropy',metrics=['accuracy']) #使用交叉熵代价函数，adam优化器优化模型，并提取准确率

# 设置训练集&测试集迭代器

In [8]:
train_generator = train_datagen.flow_from_directory( #设置训练集迭代器
    TRAIN_PATH, #训练集存放路径
    target_size=(IMG_W,IMG_H), #训练集图片尺寸
    batch_size=BATCH_SIZE #训练集批次
    )

test_generator = test_datagen.flow_from_directory( #设置测试集迭代器
    TEST_PATH, #测试集存放路径
    target_size=(IMG_W,IMG_H), #测试集图片尺寸
    batch_size=BATCH_SIZE, #测试集批次
    )

print(train_generator.class_indices) #打印迭代器分类

Found 12 images belonging to 2 classes.
Found 8 images belonging to 2 classes.
{'科比': 0, '詹姆斯': 1}


In [9]:
try:
    model = load_model('{}.h5'.format(SAVE_PATH))  #尝试读取训练好的模型，再次训练
    print('model upload,start training!')
except:
    print('not find model,start training') #如果没有训练过的模型，则从头开始训练

not find model,start training


# 训练

In [12]:
model.fit_generator( #模型拟合
                    train_generator,  #训练集迭代器
                    steps_per_epoch=len(train_generator), #每个周期需要迭代多少步（图片总量/批次大小=11200/64=175）
                    epochs=EPOCHS, #迭代周期
                    validation_data=test_generator, #测试集迭代器
                    validation_steps=len(test_generator) #测试集迭代多少步
                    )

Epoch 1/50
1/1 [==============================] - 5s 5s/step - loss: 0.6793 - acc: 0.5833 - val_loss: 0.6884 - val_acc: 0.5000
Epoch 2/50
1/1 [==============================] - 4s 4s/step - loss: 0.7397 - acc: 0.4167 - val_loss: 0.6932 - val_acc: 0.5000
Epoch 3/50
1/1 [==============================] - 4s 4s/step - loss: 0.6818 - acc: 0.5833 - val_loss: 0.6829 - val_acc: 0.3750
Epoch 4/50
1/1 [==============================] - 5s 5s/step - loss: 0.6839 - acc: 0.5833 - val_loss: 0.6811 - val_acc: 0.6250
Epoch 5/50
1/1 [==============================] - 6s 6s/step - loss: 0.6106 - acc: 0.6667 - val_loss: 0.6805 - val_acc: 0.6250
Epoch 6/50
1/1 [==============================] - 5s 5s/step - loss: 0.6276 - acc: 0.5833 - val_loss: 0.6762 - val_acc: 0.6250
Epoch 7/50
1/1 [==============================] - 5s 5s/step - loss: 0.6105 - acc: 0.7500 - val_loss: 0.6745 - val_acc: 0.6250
Epoch 8/50
1/1 [==============================] - 5s 5s/step - loss: 0.5743 - acc: 0.7500 - val_loss: 0.6801 - 

# 保存模型

In [13]:
model.save('face.h5'.format(SAVE_PATH)) #保存模型
print('finish {} epochs!'.format(EPOCHS))

finish 50 epochs!
